<a href="https://colab.research.google.com/github/imambaguss/tugasminggu_4/blob/main/HD_cleveland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Pengumpulan Data

1. Masuk ke laman UCI Machine Learning Repository atau https://archive.ics.uci.edu/
2. Cari dan unduh dataset heart disease
3. Dari banyak file di dalam dataset heart disease, kita menggunakan cleveland.data untuk diolah dan heart-disease.names untuk penjelasan dan arahan dari pengunggah dataset

# 2) Menelaah Data

Di sini dataset awal berisi dataset yang tidak rapi, tidak ada baris dan kolom. Untuk mempermudah membaca dan menganalisis data, kita gunakan itertools untuk membuat kolom. Satu barisnya memiliki 76 kolom.

In [18]:
import pandas as pd
import numpy as np
import re
import itertools

In [19]:
direct = '/content/sample_data/cleveland.data'

In [20]:
with open(direct, encoding='Latin1') as file:
  lines= [line.strip() for line in file]

lines[0:10]

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/cleveland.data'

In [ ]:
data = itertools.takewhile(
    lambda x: len(x) == 76,
    (' '.join(lines[i:(i+10)]).split() for i in range(0, len(lines), 10))
)

df = pd.DataFrame.from_records(data)

df.head()

In [ ]:
df.info()

In [ ]:
df = df.iloc[:,:-1]
df = df.drop(df.columns[0], axis =1)

In [ ]:
df = df.astype(float)
df.info()

# 3) Validasi Data

Untuk memvalidasi data, kita ubah terlebih dahulu nilai -9.0 (yaitu nilai null), menjadi value null atau NaN dengan metode `df.replace`. Setelah mengubah nilai null, kita cek lagi dengan `df.info()` untuk melihat informasi seperti apa saja kolom yang ada, berapa jumlah nilai pada tiap kolom, dan type data masing-masing kolom.



In [ ]:
# Mengganti nilai -9.0 menjadi null atau NaN karena -9.0 adalah nilai null
df.replace(-9.0, np.nan, inplace=True)
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.info()

# 4) Menentukan Objek Data

Dalam heart-disease.names sudah ditentukan kolom mana saja yang akan digunakan nanti, sisanya akan di-drop.
Kolom atau atribut yang digunakan:

-- 1. #3  (age)       
-- 2. #4  (sex)       
-- 3. #9  (cp)        
-- 4. #10 (trestbps)  
-- 5. #12 (chol)      
-- 6. #16 (fbs)       
-- 7. #19 (restecg)   
-- 8. #32 (thalach)   
-- 9. #38 (exang)     
-- 10. #40 (oldpeak)   
-- 11. #41 (slope)     
-- 12. #44 (ca)        
-- 13. #51 (thal)      
-- 14. #58 (num)  

Kolom2 yang digunakan ini penting untuk perhitungan kondisi jantung nantinya

In [ ]:
df_selected = df.iloc[:, [1, 2, 7, 8, 10, 14, 17, 30, 36, 38, 39, 42, 49, 56]]
df_selected.head()

In [ ]:
df_selected.info()

In [ ]:
column_mapping = {
    2: 'age',
    3: 'sex',
    8: 'cp',
    9: 'trestbps',
    11: 'chol',
    15: 'fbs',
    18: 'restecg',
    31: 'thalach',
    37: 'exang',
    39: 'oldpeak',
    40: 'slope',
    43: 'ca',
    50: 'thal',
    57: 'target'
}
df_selected.rename(columns=column_mapping, inplace=True)
df_selected.info()

In [ ]:
df_selected.value_counts()

# 5) Membersihkan Data

langkah2 yang diambil untuk membersihkan data:
1. Cek tiap kolom apakah ada nilai null, jika ada maka drop kolom yang terdapat nilai null. Biasanya kolom yang memiliki 100 atau lebih jumlah nilai yang null.
2. Jika hanya sedikit nilai null-nya, lakukan perhitungan nilai mean pada kolom yang ada nilai null-nya. Pertama drop terlebih dahulu kolom yang ingin dicari nilai mean-nya. Kedua ubah tipe datanya menjadi float. Ketiga hitung nilai mean beserta dibulatkan. Keempat masukan kembali kolom tadi, ke dalam kolom utama.

In [ ]:
df_selected.isnull().sum()

In [ ]:
df_selected.head(10)

In [ ]:
# Drop kolom yg ingin dihitung nilai mean-nya
meanCa = df_selected['ca'].dropna()
meanThal = df_selected['thal'].dropna()

# Ubah ke tipe float
meanCa = meanCa.astype(float)
meanThal = meanThal.astype(float)

# Bulatkan bilangannya
meanCa = round(meanCa.mean())
meanThal = round(meanThal.mean())

In [ ]:
fill_values = {'ca': meanCa, 'thal': meanThal}
dfClean = df_selected.fillna(value=fill_values)
dfClean.info()

In [ ]:
dfClean.isnull().sum()

In [ ]:
duplicate_rows = dfClean.duplicated()
dfClean[duplicate_rows]

In [ ]:
dfClean.head(10)

In [ ]:
dfClean['target'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Mencari korelasi antar fitur
dfClean.corr()

In [21]:
cor_mat = dfClean.corr()
fig,ax = plt.subplots(figsize=(15,10))
sns.heatmap(cor_mat, annot=True, linewidths=0.5, fmt=".3f")

NameError: name 'dfClean' is not defined

# 6) Konstruksi Data

Dalam tahap ini Konstruksi data salah satu tujuannya yaitu untuk menyesuaikan semua tipe data yang ada di dalam dataset. Namun pada tahap ini dataset sudah memiliki tipe data yang sesuai sehingga tidak perlu dilakukan penyesuaian kembali dan tidak ada penambahan fitur untuk meningkatkan performa model

# 7) Menentukan Label Data

Menentukan label data dilakukan untuk memisahkan kolom atribut dan kolom target/class/label. Kita pisahkan terlebih dahulu kolomnya untuk x adalah kolom yang akan dilakukan perhitungan, dan y adalah kolom target. Kolom target ini akan dicek apakah jumlah labelnya mengalami imbalance atau jumlah nilai yang tidak seimbang pada tiap labelnya. Jika mengalami imbalance, maka kita gunakan metode oversampling dengan algoritma SMOTE. Dan nanti hasilnya jumlah setiap label akan sama rata.

In [ ]:
dfClean.info()

In [ ]:
X = dfClean.drop("target", axis=1).values
y = dfClean.iloc[:, -1]

# Visualisasi target
dfClean['target'].value_counts().plot(kind='bar',figsize=(10,6),color=['green','blue'])
plt.title("Count of the target")
plt.xticks(rotation=0);

In [ ]:
from imblearn.over_sampling import SMOTE

# oversampling
smote = SMOTE(random_state=42)
X_smote_resampled, y_smote_resampled = smote.fit_resample(X, y)

In [ ]:
# Membuat visual plot target sebelum SMOTE
plt.subplot(1, 2, 1)
new_df1 = pd.DataFrame(data=y)

plt.subplot(1, 2, 1)
new_df1.value_counts().plot(kind='bar',figsize=(10,6),color=['green', 'blue', 'red', 'yellow'])
plt.title("target before over sampling with SMOTE ")
plt.xticks(rotation=0);

# Membuat visual plot target setelah SMOTE
plt.subplot(1, 2, 2)
new_df2 = pd.DataFrame(data=y_smote_resampled)

new_df2.value_counts().plot(kind='bar',figsize=(10,6),color=['green','blue','red','yellow'])
plt.title("target after over sampling with SMOTE")
plt.xticks(rotation=0);

plt.tight_layout()
plt.show()

In [ ]:
new_df1 = pd.DataFrame(data=y)
print("Sebelum SMOTE :")
new_df1.value_counts()

In [22]:
print("\nSetelah SMOTE :")
new_df2 = pd.DataFrame(data=y_smote_resampled)
new_df2.value_counts()


Setelah SMOTE :


NameError: name 'y_smote_resampled' is not defined

In [ ]:
dfClean.describe()

Pada deskripsi diatas dapat dilihat bahwa terdapat rentang nilai yang cukup jauh pada standar deviasi setiap fitur dataset yang kita miliki. Oleh karena itu perlu dilakukan normalisasi/standarisasi agar memperkecil rentang antara standar deviasi setiap kolom.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_smote_resampled_normal = scaler.fit_transform(X_smote_resampled)

In [ ]:
len(X_smote_resampled_normal)

In [ ]:
dfcek1 = pd.DataFrame(X_smote_resampled_normal)
dfcek1.describe()

Setelah dilakukan normalisasi pada fitur, selanjutnya kita perlu membagi fitur dan target menjadi data train dan test.

# 8) Model

Pemisahan data training dan data testing dilakukan 2 kali, yang pertama untuk oversample saja, dan kedua untuk oversample + normalisasi.
Perbandingan yang digunakan adalah 80:20, 80% untuk data training, dan 20% untuk data testing.

Model yang digunakan adalah k-Nearest Neighbours, Random Forest, dan XGBoost

Ada 3 metode dan masing2 metode menggunakan model yang sama, yg pertama adalah Oversample, kedua oversample + normalisasi, dan ketiga oversample + normalisai + Tunning.

Masing2 metode akan dihitung confusion matrix-nya (akurasi, presisi, recall, dan f1-score)

In [ ]:
from sklearn.model_selection import train_test_split

# membagi fitur dan target menjadi data train dan test (untuk yang oversample saja)
X_train, X_test, y_train, y_test = train_test_split(X_smote_resampled, y_smote_resampled, test_size=0.2, random_state=42,stratify=y_smote_resampled)

# membagi fitur dan target menjadi data train dan test (untuk yang oversample + normalization)
X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(X_smote_resampled_normal, y_smote_resampled, test_size=0.2, random_state=42,stratify = y_smote_resampled)

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,roc_auc_score,confusion_matrix,precision_score

def evaluation(Y_test,Y_pred):
  acc = accuracy_score(Y_test,Y_pred)
  rcl = recall_score(Y_test,Y_pred,average = 'weighted')
  f1 = f1_score(Y_test,Y_pred,average = 'weighted')
  ps = precision_score(Y_test,Y_pred,average = 'weighted')
  metric_dict={'accuracy': round(acc,3),
    'recall': round(rcl,3),
    'F1 score': round(f1,3),
    'Precision score': round(ps,3)
  }
  return print(metric_dict)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

knn_model = KNeighborsClassifier(n_neighbors = 3)
knn_model.fit(X_train, y_train)

In [ ]:
y_pred_knn = knn_model.predict(X_test)

# Evaluate the KNN model
print("K-Nearest Neighbors (KNN) Model:")
accuracy_knn_smote = round(accuracy_score(y_test,y_pred_knn),3)
print("Accuracy:", accuracy_knn_smote)
print("Classification Report:")
print(classification_report(y_test, y_pred_knn))

In [ ]:
evaluation(y_test,y_pred_knn)

Pada visualisasi ini ditampilkan visualisasi confusion matrix untuk membandingkan hasil prediksi model dengan nilai sebenarnya.

In [ ]:
cm = confusion_matrix(y_test, y_pred_knn)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# Random Forest

Selanjutnya kita akan membangun model dengan algoritma random forest dengan n_estimators yaitu 100, n_estimators sendiri berguna mengatur jumlah pohon keputusan yang akan dibangun

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_model.predict(X_test)

# Evaluate the Random Forest model
print("\nRandom Forest Model:")
accuracy_rf_smote = round(accuracy_score(y_test, y_pred_rf),3)
print("Accuracy:",accuracy_rf_smote)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
evaluation(y_test,y_pred_rf)

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# XGBoost

Pada tahap ini dalam membangun model, kita akan menggunakan algoritma XGBoost dengan learning rate yaitu 0.1. learning rate berguna untuk mengontrol seberapa besar kita menyesuaikan bobot model.

In [ ]:
xgb_model = XGBClassifier(learning_rate=0.1, n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the XGBoost model
print("\nXGBoost Model:")
accuracy_xgb_smote = round(accuracy_score(y_test, y_pred_xgb),3)
print("Accuracy:",accuracy_xgb_smote)
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb))

In [ ]:
evaluation(y_test,y_pred_xgb)

In [ ]:
cm = confusion_matrix(y_test, y_pred_xgb)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# Oversample + Normalisasi

Pada bagian ini kita akan membuat sebuah model yang dimana data yang dipakai kali ini yang sudah dilakukan oversample dan normalisasi.
Algoritma yang digunakan sama seperti sebelumnya yaitu KNN, Random Forest, dan XGBoost. Sekaligus dibuat visualisasi hasil evaluasi pada
masing-masing model.

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train_normal, y_train_normal)

In [ ]:
y_pred_knn = knn_model.predict(X_test_normal)

# Evaluate the KNN model
print("K-Nearest Neighbors (KNN) Model:")
accuracy_knn_smote_normal = round(accuracy_score(y_test_normal,y_pred_knn),3)
print("Accuracy:", accuracy_knn_smote_normal)
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_knn))

In [ ]:
evaluation(y_test_normal,y_pred_knn)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_knn)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_normal, y_train_normal)

In [ ]:
y_pred_rf = rf_model.predict(X_test_normal)

# Evaluate the Random Forest model
print("\nRandom Forest Model:")
accuracy_rf_smote_normal = round(accuracy_score(y_test_normal, y_pred_rf),3)
print("Accuracy:",accuracy_rf_smote_normal )
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_rf))

In [ ]:
evaluation(y_test_normal,y_pred_rf)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# XGBoost

In [ ]:
xgb_model = XGBClassifier(learning_rate=0.1, n_estimators=100, random_state=42)
xgb_model.fit(X_train_normal, y_train_normal)

In [ ]:
y_pred_xgb = xgb_model.predict(X_test_normal)

# Evaluate the XGBoost model
print("\nXGBoost Model:")
accuracy_xgb_smote_normal = round(accuracy_score(y_test_normal, y_pred_xgb),3)
print("Accuracy:",accuracy_xgb_smote_normal)
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_xgb))

In [ ]:
evaluation(y_test_normal,y_pred_xgb)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_xgb)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# Oversample + Normalisasi + Tunning

Pada pembuatan model kali ini masih menggunakan algoritma yang sama (KNN, Random Forest, dan XGBoost), namun data yang digunakan
adalah data yang sudah dilakukan TunNIng Parameter, Normalisasi, dan Oversample.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV

**Setiap parameter tunnning tidak selalu sama karena bergantung pada algoritma yang digunakan.**

# KNN

In [ ]:
knn_model = KNeighborsClassifier()

param_grid = {
  "n_neighbors": range(3, 21),
  "metric": ["euclidean", "manhattan", "chebyshev"],
  "weights": ["uniform", "distance"],
  "algorithm": ["auto", "ball_tree", "kd_tree"],
  "leaf_size": range(10, 61),
}

knn_model = RandomizedSearchCV(estimator=knn_model, param_distributions=param_grid, n_iter=100, scoring="accuracy", cv=5)

knn_model.fit(X_train_normal, y_train_normal)

best_params = knn_model.best_params_
print(f"Best parameters: {best_params}")

In [ ]:
y_pred_knn = knn_model.predict(X_test_normal)

# Evaluate the KNN model
print("K-Nearest Neighbors (KNN) Model:")
accuracy_knn_smote_normal_Tun = round(accuracy_score(y_test_normal,y_pred_knn),3)
print("Accuracy:", accuracy_knn_smote_normal_Tun)
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_knn))

In [ ]:
evaluation(y_test_normal,y_pred_knn)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_knn)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# Random Forest

In [ ]:
rf_model = RandomForestClassifier()

param_grid = {
  "n_estimators": [100, 200],
  "max_depth": [ 10, 15],
  "min_samples_leaf": [1, 2],
  "min_samples_split": [2, 5],
  "max_features": ["sqrt", "log2"],
  # "random_state": [42, 100, 200]
}

rf_model = RandomizedSearchCV(rf_model, param_grid, n_iter=100, cv=5, n_jobs=-1)

rf_model.fit(X_train_normal, y_train_normal)

best_params = rf_model.best_params_
print(f"Best parameters: {best_params}")

In [ ]:
y_pred_rf = rf_model.predict(X_test_normal)

# Evaluate the Random Forest model
print("\nRandom Forest Model:")
accuracy_rf_smote_normal_Tun = round(accuracy_score(y_test_normal, y_pred_rf),3)
print("Accuracy:",accuracy_rf_smote_normal_Tun)
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_rf))

In [ ]:
evaluation(y_test_normal,y_pred_rf)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_knn)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# XGBoost

In [ ]:
xgb_model = XGBClassifier()

param_grid = {
  "max_depth": [3, 5, 7],
  "learning_rate": [0.01, 0.1],
  "n_estimators": [100, 200],
  "gamma": [0, 0.1],
  "colsample_bytree": [0.7, 0.8],
}

xgb_model = RandomizedSearchCV(xgb_model, param_grid, n_iter=10, cv=5, n_jobs=-1)

xgb_model.fit(X_train_normal, y_train_normal)

best_params = xgb_model.best_params_
print(f"Best parameters: {best_params}")

In [ ]:
y_pred_xgb = xgb_model.predict(X_test_normal)

# Evaluate the XGBoost model
print("\nXGBoost Model:")
accuracy_xgb_smote_normal_Tun = round(accuracy_score(y_test_normal, y_pred_xgb),3)
print("Accuracy:",accuracy_xgb_smote_normal_Tun)
print("Classification Report:")
print(classification_report(y_test_normal, y_pred_xgb))

In [ ]:
evaluation(y_test_normal,y_pred_xgb)

In [ ]:
cm = confusion_matrix(y_test_normal, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('True')
plt.ylabel('Predict')
plt.show()

# 9) Evaluasi

Selanjutnya kita akan melakukan evaluasi data sekaligus membandingkan antar algoritma guna dengan tujuan mengetahui jenis model
algoritma yang menghasilkan hasil akurasi terbaik.

In [ ]:
import matplotlib.pyplot as plt

model_comp1 = pd.DataFrame({'Model': ['K-Nearest Neighbour','Random Forest',
                                      'XGBoost'], 'Accuracy': [accuracy_knn_smote*100,
                                                               accuracy_rf_smote*100,accuracy_xgb_smote*100]})

model_comp1.head()

In [ ]:
# Membuat bar plot dengan keterangan jumlah
fig, ax = plt.subplots()
bars = plt.bar(model_comp1['Model'], model_comp1['Accuracy'], color=['red', 'green', 'blue'])
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.title('Oversample')
plt.xticks(rotation=45, ha='right') # Untuk memutar label sumbu x agar lebih mudah dibaca

# Menambahkan keterangan jumlah di atas setiap bar
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

plt.show()

In [ ]:
model_comp2 = pd.DataFrame({'Model': ['K-Nearest Neighbour','Random Forest',
                                      'XGBoost'], 'Accuracy': [accuracy_knn_smote_normal*100,
                                                               accuracy_rf_smote_normal*100,accuracy_xgb_smote_normal*100]})

model_comp2.head()

In [ ]:
# Membuat bar plot dengan keterangan jumlah
fig, ax = plt.subplots()
bars = plt.bar(model_comp2['Model'], model_comp2['Accuracy'], color=['red', 'green', 'blue'])
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.title('Normalization + Oversampling')
plt.xticks(rotation=45, ha='right') # Untuk memutar label sumbu x agar lebih mudah dibaca

# Menambahkan keterangan jumlah di atas setiap bar
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

plt.show()

In [ ]:
model_comp3 = pd.DataFrame({'Model': ['K-Nearest Neighbour','Random Forest',
                                      'XGBoost'], 'Accuracy': [accuracy_knn_smote_normal_Tun*100,
                                                               accuracy_rf_smote_normal_Tun*100,accuracy_xgb_smote_normal_Tun*100]})

model_comp3.head()

In [ ]:
# Membuat bar plot dengan keterangan jumlah
fig, ax = plt.subplots()
bars = plt.bar(model_comp3['Model'], model_comp3['Accuracy'], color=['red', 'green', 'blue'])
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.title('Normalization + Oversampling + Tunning')
plt.xticks(rotation=45, ha='right') # Untuk memutar label sumbu x agar lebih mudah dibaca

# Menambahkan keterangan jumlah di atas setiap bar
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

plt.show()

In [ ]:
# Data frame
model_compBest = pd.DataFrame({
    'Model': ['K-Nearest Neighbour OverSample Tunning', 'Random Forest OverSample',
              'XGB OverSample Standarization Tunning'],
    'Accuracy': [accuracy_knn_smote_normal_Tun*100, accuracy_rf_smote_normal*100,
                 accuracy_xgb_smote_normal_Tun*100]
})

# Membuat bar plot dengan keterangan jumlah
fig, ax = plt.subplots()
bars = plt.bar(model_compBest['Model'], model_compBest['Accuracy'], color=['red', 'green', 'blue'])
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.title('Best Model Comparison')
plt.xticks(rotation=45, ha='right') # Untuk memutar label sumbu x agar lebih mudah dibaca

# Menambahkan keterangan jumlah di atas setiap bar
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')
plt.show()

# 10) Kesimpulan

Penelitian di atas menggunakan model KNN, Random Forest, dan XGBoost. Sebelum melakukan modeling, terjadi imbalance pada dataset dan harus diperbaiki. Untuk memperbaikinya dengan metode oversampling algoritma SMOTE, hasilnya jumlah nilai pada setiap label sama rata semua.

Tingkat akurasi yang didapat tiap model adalah:
1.   Oversample
*   KNN : 70%
*   Random Forest : 94%
*   XGBoost : 88%

2.   Oversample + Normalisasi
*   KNN : 82%
*   Random Forest : 94%
*   XGBoost : 88%

3.   Oversample + Normalisasi + Tunning
*   KNN : 89%
*   Random Forest : 94%
*   XGBoost : 89%

